<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Deep Learning for NLP
  </div> 
  
<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Part I - 3 <br><br><br>
  Language Modeling
  </div> 

  <div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 20px; 
      text-align: center; 
      padding: 15px;">
  </div> 

  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  Jean-baptiste AUJOGUE
  </div> 

### Part I

1. Word Embedding

2. Sentence Classification

    _Applications :_
    
    - Extractive Summarization
    - Sentiment Analysis
    - Text segmentation


3. <font color=red>**Language Modeling**</font>

4. Sentence tagging

    _Applications :_
    
    - Part-of-speech Tagging
    - Named Entity Recognition
    - Automatic Value Extraction
    


### Part II

5. Auto-Encoding

6. Machine Translation

7. Text Classification




### Part III

8. Abstractive Summarization

9. Question Answering

10. Chatbot


</div>

***

<a id="plan"></a>

# Overview

The global structure of the [language model](#language_model) is the pipeline of two modules, followed by a final classification layer :



| | Module |  | |
|------|------|------|------|
| 1 | **Word Embedding** | [I.1 Custom model](#word_level_custom) | [I.2 Gensim Model](#gensim) | [I.3 FastText model](#fastText) |
| 2 | **Contextualization** | [II.1 bidirectionnal GRU](#bi_gru) | [II.2 Transformer](#transformer) |
| 3 | **Attention** | [III.1 Attention](#attention) | [III.2 Multi-head Attention](#attention) |



All details on Word Embedding modules and their pre-training are found in **Part I - 1**.

Exemples d'implémentation en PyTorch :

- https://github.com/pytorch/examples/blob/master/word_language_model/model.py


Différentes architectures sont décrites dans la litérature :

- Regularizing and Optimizing LSTM Language Models - https://arxiv.org/pdf/1708.02182.pdf

Un modèle linguistique est intérressant en soi, mais peut aussi servir pour le pré-entrainement de couches basses d'un modèle plus complexe :

- Deep contextualized word representations - https://arxiv.org/pdf/1802.05365.pdf
- Improving Language Understanding by Generative Pre-Training - https://www.cs.ubc.ca/~amuham01/LING530/papers/radford2018improving.pdf
- Language Models are Unsupervised Multitask Learners - https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf

# Packages

In [1]:
import sys
import warnings
from __future__ import unicode_literals, print_function, division
import os
from io import open
import unicodedata
import string
import time
import math
import re
import random
import pickle
import copy
from unidecode import unidecode
import itertools
import matplotlib
import matplotlib.pyplot as plt


# for special math operation
from sklearn.preprocessing import normalize


# for manipulating data 
import numpy as np
#np.set_printoptions(threshold=np.nan)
import pandas as pd
import bcolz # see https://bcolz.readthedocs.io/en/latest/intro.html
import pickle


# for text processing
import gensim
from gensim.models import KeyedVectors
#import spacy
import nltk
#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem.porter import PorterStemmer


# for deep learning
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


warnings.filterwarnings("ignore")
print('python version :', sys.version)
print('pytorch version :', torch.__version__)
print('DL device :', device)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


python version : 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
pytorch version : 0.4.0
DL device : cuda


In [2]:
path_to_NLP = 'C:\\Users\\Jb\\Desktop\\NLP'

In [3]:
sys.path.append(path_to_NLP + '\\chatNLP')

# Corpus

[Back to top](#plan)

Le texte est importé et mis sous forme de liste, où chaque élément représente un texte présenté sous forme d'une liste de mots.<br> Le corpus et donc une fois importé sous le forme :<br>

- corpus = [text, label]<br>
- text   = [word]<br>
- word   = str<br>

In [24]:
def cleanSentence(sentence): # -------------------------  str
    sw = ['']
    #sw += nltk.corpus.stopwords.words('english')
    #sw += nltk.corpus.stopwords.words('french')

    def unicodeToAscii(s):
        """Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427"""
        return ''.join( c for c in unicodedata.normalize('NFD', s)
                        if unicodedata.category(c) != 'Mn')

    def normalizeString(s):
        '''Remove rare symbols from a string'''
        s = unicodeToAscii(s.lower().strip()) # 
        #s = re.sub(r"[^a-zA-Z\.\(\)\[\]]+", r" ", s)  # 'r' before a string is for 'raw' # ?&\%\_\- removed # set('''.,:;()*#&-_%!?/\'")''')
        return s

    def wordTokenizerFunction():
        # base version
        function = lambda sentence : sentence.strip().split()

        # nltk version
        #function = word_tokenize    
        return function

    # 1 - caractères spéciaux
    def clean_sentence_punct(text): # --------------  str
        text = normalizeString(text)
        # suppression de la dernière ponctuation
        if (len(text) > 0 and text[-1] in ['.', ',', ';', ':', '!', '?']) : text = text[:-1]

        text = text.replace(r'(', r' ( ')
        text = text.replace(r')', r' ) ')
        text = text.replace(r'[', r' [ ')
        text = text.replace(r']', r' ] ')
        text = text.replace(r'<', r' < ')
        text = text.replace(r'>', r' > ')

        text = text.replace(r':', r' : ')
        text = text.replace(r';', r' ; ')
        for i in range(5) :
            text = re.sub('(?P<val1>[0-9])\.(?P<val2>[0-9])', '\g<val1>__-__\g<val2>', text)
            text = re.sub('(?P<val1>[0-9]),(?P<val2>[0-9])', '\g<val1>__-__\g<val2>', text)
        text = text.replace(r',', ' , ')
        text = text.replace(r'.', ' . ')
        for i in range(5) : text = re.sub('(?P<val1>[p0-9])__-__(?P<val2>[p0-9])', '\g<val1>.\g<val2>', text)
        text = re.sub('(?P<val1>[0-9]) \. p \. (?P<val2>[0-9])', '\g<val1>.p.\g<val2>', text)
        text = re.sub('(?P<val1>[0-9]) \. s \. (?P<val2>[0-9])', '\g<val1>.s.\g<val2>', text)

        text = text.replace(r'"', r' " ')
        text = text.replace(r'’', r" ' ")
        text = text.replace(r'”', r' " ')
        text = text.replace(r'“', r' " ')
        text = text.replace(r'/', r' / ')

        text = re.sub('(…)+', ' … ', text)
        text = text.replace('≤', ' ≤ ')          
        text = text.replace('≥', ' ≥ ')
        text = text.replace('°c', ' °c ')
        text = text.replace('°C', ' °c ')
        text = text.replace('ºc', ' °c ')
        text = text.replace('n°', 'n° ')
        text = text.replace('%', ' % ')
        text = text.replace('*', ' * ')
        text = text.replace('+', ' + ')
        text = text.replace('-', ' - ')
        text = text.replace('_', ' ')
        text = text.replace('®', ' ')
        text = text.replace('™', ' ')
        text = text.replace('±', ' ± ')
        text = text.replace('÷', ' ÷ ')
        text = text.replace('–', ' - ')
        text = text.replace('μg', ' µg')
        text = text.replace('µg', ' µg')
        text = text.replace('µl', ' µl')
        text = text.replace('μl', ' µl')
        text = text.replace('µm', ' µm')
        text = text.replace('μm', ' µm')
        text = text.replace('ppm', ' ppm')
        text = re.sub('(?P<val1>[0-9])mm', '\g<val1> mm', text)
        text = re.sub('(?P<val1>[0-9])g', '\g<val1> g', text)
        text = text.replace('nm', ' nm')

        text = re.sub('fa(?P<val1>[0-9])', 'fa \g<val1>', text)
        text = re.sub('g(?P<val1>[0-9])', 'g \g<val1>', text)
        text = re.sub('n(?P<val1>[0-9])', 'n \g<val1>', text)
        text = re.sub('p(?P<val1>[0-9])', 'p \g<val1>', text)
        text = re.sub('q_(?P<val1>[0-9])', 'q_ \g<val1>', text)
        text = re.sub('u(?P<val1>[0-9])', 'u \g<val1>', text)
        text = re.sub('ud(?P<val1>[0-9])', 'ud \g<val1>', text)
        text = re.sub('ui(?P<val1>[0-9])', 'ui \g<val1>', text)

        text = text.replace('=', ' ')
        text = text.replace('!', ' ')
        text = text.replace('-', ' ')
        text = text.replace(r' , ', ' ')
        text = text.replace(r' . ', ' ')

        text = re.sub('(?P<val>[0-9])ml', '\g<val> ml', text)
        text = re.sub('(?P<val>[0-9])mg', '\g<val> mg', text)

        for i in range(5) : text = re.sub('( [0-9]+ )', ' ', text)
        #text = re.sub('cochran(\S)*', 'cochran ', text)
        return text

    # 3 - split des mots
    def wordSplit(sentence, tokenizeur): # ------------- [str]
        return tokenizeur(sentence)

    # 4 - mise en minuscule et enlèvement des stopwords
    def stopwordsRemoval(sentence, sw): # ------------- [[str]]
        return [word for word in sentence if word not in sw]

    # 6 - correction des mots
    def correction(text):
        def correct(word):
            return spelling.suggest(word)[0]
        list_of_list_of_words = [[correct(word) for word in sentence] for sentence in text]
        return list_of_list_of_words

    # 7 - stemming
    def stemming(text): # ------------------------- [[str]]
        list_of_list_of_words = [[PorterStemmer().stem(word) for word in sentence if word not in sw] for sentence in text]
        return list_of_list_of_words


    tokenizeur = wordTokenizerFunction()
    sentence = clean_sentence_punct(str(sentence))
    sentence = wordSplit(sentence, tokenizeur)
    sentence = stopwordsRemoval(sentence, sw)
    #text = correction(text)
    #text = stemming(text)
    return sentence


def importSheet(file_name) :
    def cleanDatabase(db):
        words = []
        title = ''
        for pair in db :
            #print(pair)
            current_tile = pair[0].split(' | ')[-1]
            if current_tile != title :
                words += cleanSentence(current_tile) #[str]
                title  = current_tile                # str
            words += cleanSentence(str(pair[1]).split(' | ')[-1])     #[str]
        return words

    df = pd.read_excel(file_name, sep = ',', header = None)
    headers = [i for i, titre in enumerate(df.ix[0,:].values) if i in [1, 2] or titre == 'score manuel'] 
    db = df.ix[1:, headers].values.tolist()
    db = [el[:2] for el in db if el[-1] in [0,1, 10]]
    words = cleanDatabase(db)
    return words


def importCorpus(path_to_data) :
    corpus = []
    reps = os.listdir(path_to_data)
    for rep in reps :
        files = os.listdir(path_to_data + '\\' + rep)
        for file in files :
            file_name = path_to_data + '\\' + rep + '\\' + file
            corpus.append(importSheet(file_name))
    return corpus

In [25]:
corpus = importCorpus(path_to_NLP + '\\data\\AMM')

# 1 Modules

## 1.1 Word Embedding module

[Back to top](#plan)

We assume that embedding model were pretrained following the steps detailed in **Part I - 1**.<br>
We consider here Word2Vec models pre-trained following the Skip-Gram training objective.

<a id="word_level_custom"></a>


#### 1.1.1 Custom model

The model is frozen after being loaded, so that none of its parameters is targeted by the sentence classifier optimizer.

In [7]:
from chatNLP.models.Word_Embedding import Word2Vec as myWord2Vec
from chatNLP.models.Word_Embedding import Word2VecConnector
from chatNLP.utils.Lang import Lang

In [343]:
custom_word2vec = torch.load(path_to_NLP + '\\saves\\models\\DL4NLP_I1_skipgram.pt').freeze()

<a id="gensim"></a>

#### 1.1.2 Gensim model


In [9]:
from gensim.models import Word2Vec
from gensim.test.utils import datapath, get_tmpfile

In [254]:
gensim_word2vec = Word2VecConnector(Word2Vec.load(get_tmpfile(path_to_NLP + "\\saves\\models\\DL4NLP_I1_skipgram_gensim.model")))

<a id="fastText"></a>

#### 1.1.3 FastText model

In [12]:
from gensim.models.fasttext import FastText
from gensim.test.utils import datapath, get_tmpfile

In [320]:
fastText_word2vec = Word2VecConnector(FastText.load(get_tmpfile(path_to_NLP + "\\saves\\models\\DL4NLP_I1_fasttext.model")))

## 1.2 Contextualization module

[Back to top](#plan)

The contextualization layer transforms a sequences of word vectors into another one, of same length, where each output vector corresponds to a new version of each input vector that is contextualized with respect to neighboring vectors.

<a id="bi_gru"></a>

#### 1.2.1 Bi-directionnal GRU contextualization

This module consists of a bi-directional _Gated Recurrent Unit_ (GRU) that supports packed sentences :

In [ ]:
from chatNLP.modules import RecurrentEncoder

<a id="language_model"></a>

# 2 Language Model

[Back to top](#plan)


### Model

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self, device, 
                 lang,
                 embedding_weights, 
                 hidden_dim,
                 n_layers = 1, 
                 dropout = 0): 
        
        super(LanguageModel, self).__init__()
        # relevant quantities
        self.device = device
        self.lang = lang
        self.hidden_dim = hidden_dim
        self.dropout_p = dropout
        self.n_layers = n_layers               # number of stacked GRU layers
        self.output_dim = hidden_dim           # dimension of outputed rep. of words and utterance
        # parameters
        self.embedding = nn.Embedding(embedding_weights[0], embedding_weights[1]) if type(embedding_weights) == tuple else \
                         nn.Embedding.from_pretrained(torch.FloatTensor(embedding_weights), freeze=True)
        for p in self.embedding.parameters() :
            embedding_dim = p.data.size(1)
        self.dropout = nn.Dropout(p = dropout)
        self.gru = nn.GRU(embedding_dim, 
                          self.hidden_dim, 
                          n_layers,
                          dropout=(0 if n_layers == 1 else dropout), 
                          bidirectional = False)
        self.out = nn.Linear(self.hidden_dim, lang.n_words)
        #self.out.weight = self.embedding.weight
        
        
    # ------------- technical methods ------------------    
    def nbParametres(self) :
        count = 0
        for p in self.parameters():
            if p.requires_grad == True :
                count += p.data.nelement()
        return count
    

    def variableFromSentence(self, sentence):
        '''Turn a sentence into a torch variable, containing a list of indices according
           to a given language.'''
        indexes = [self.lang.word2index[word] for word in sentence if word in self.lang.word2index]          
        result = Variable(torch.LongTensor(indexes).view(-1, 1)).to(self.device)
        return result
    

    def initHidden(self): 
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_dim)).to(self.device)
    
    
    # ------------- main methods ------------------
    def generateWord(self, utterance, hidden = None):
        embeddings = self.embedding(utterance)                          # dim = (input_length, 1, embedding_dim)
        embeddings = self.dropout(embeddings)                           # dim = (input_length, 1, embedding_dim)
        hidden = self.initHidden() if hidden is None else hidden
        _, hidden = self.gru(embeddings, hidden)                        # dim = (input_length, 1, hidden_dim)
        hidden = self.dropout(hidden)
        log_probs = F.log_softmax(self.out(hidden[-1]))         # dim = (1, lang_size)
        return log_probs, hidden  

    
    def forward(self, sentence = ['SOS'], hidden = None, limit = 10, retain = False):
        result = sentence + ['\033[94m']
        count = 0
        stop = False
        utterance = self.variableFromSentence(sentence)
        while not stop :
            log_probs, hidden = self.generateWord(utterance, hidden)
            topv, topi = log_probs[0].data.topk(1)
            ni = topi.item()
            utterance = Variable(torch.LongTensor([[ni]])).to(self.device)
            result.append(self.lang.index2word[ni])
            count += 1
            if count == limit or (type(limit) == str and ni == self.lang.word2index[limit]) or count == 50 :
                stop = True
        if not retain :
            print(' '.join(result + ['\033[0m']))
            return
        else :
            return result, hidden   
        
        
class LanguageModelTrainer(object):
    def __init__(self, 
                 device,
                 criterion = nn.NLLLoss(reduce = False), #nn.BCEWithLogitsLoss(), #nn.BCELoss(), 
                 optimizer = optim.SGD,
                 print_every=100):
        # relevant quantities
        self.device = device
        self.criterion = criterion
        self.optimizer = optimizer
        self.print_every = print_every
        
        
    def asMinutes(self, s):
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)
    
    
    def timeSince(self, since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return '%s (- %s)' % (self.asMinutes(s), self.asMinutes(rs))
        
        
    def trainLoop(self, agent, sentence, word, optimizer, learning_rate):
        """Performs a training loop, with forward pass and backward pass for gradient optimisation."""
        optimizer.zero_grad()
        agent.zero_grad()
        word_var     = agent.variableFromSentence([word]).view(-1)
        sentence_var = agent.variableFromSentence(sentence)
        log_probs, _ = agent.generateWord(sentence_var)
        
        loss = self.criterion(log_probs, word_var)
        topv, topi = log_probs[0].data.topk(1)
        loss_diff = 1 if topi.item() != word_var.data.item() else 0
        loss.backward()
        optimizer.step()                                
        return loss.data[0], loss_diff
        
        
    def train(self,
              agent, 
              sentences,
              n_iters = 100,
              learning_rate=0.01,
              random_state = 42
             ):
        """Performs training over a given dataset and along a specified amount of loops."""
        np.random.seed(random_state)
        start = time.time()
        optimizer = self.optimizer([param for param in agent.parameters() if param.requires_grad == True], lr=learning_rate)
        print_loss_total = 0  
        print_loss_diff_mots_total = 0
        for iter in range(1, n_iters + 1):
            sentence = random.choice(sentences) #--- [str]
            i = random.choice(range(len(sentence)))
            context = ['SOS'] + sentence[:i] 
            target = sentence[i]

            loss, loss_diff_mots = self.trainLoop(agent, context, target, optimizer, learning_rate)
            
            # affichage
            print_loss_total += loss
            print_loss_diff_mots_total += loss_diff_mots       
            if iter % (self.print_every) == 0:
                print_loss_avg = print_loss_total / self.print_every
                print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                print_loss_total = 0
                print_loss_diff_mots_total = 0
                print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters),
                                             iter, iter / n_iters * 100, 
                                                  print_loss_avg, print_loss_diff_mots_avg))

In [424]:
class LanguageModel(nn.Module) :
    def __init__(self, device, word2vec, hidden_dim, n_layers, dropout = 0, optimizer = optim.SGD) :
        super(LanguageModel, self).__init__()
        
        # embedding
        self.word2vec  = word2vec
        self.context   = RecurrentEncoder(self.word2vec.output_dim, hidden_dim, n_layers, dropout, bidirectional = False)
        self.out       = nn.Linear(self.context.output_dim, self.word2vec.lang.n_words)
        self.act       = F.softmax
        
        # optimizer
        self.criterion = nn.NLLLoss(size_average = False)
        self.optimizer = optimizer
        
        # load to device
        self.device = device
        self.to(device)
        
    def nbParametres(self) :
        return sum([p.data.nelement() for p in self.parameters() if p.requires_grad == True])
    
    def forward(self, sentence = 'SOS', hidden = None, limit = 10):
        words  = sentence.split(' ')
        result = words + ['\033[94m']
        hidden, count, stop = None, 0, False
        while not stop :
            # compute probs
            embeddings = self.word2vec(words, self.device)
            _, hidden  = self.context(embeddings, hidden)
            probs      = self.act(self.out(hidden[:, -1, :])).view(-1)
            # get predicted word
            topv, topi = probs.data.topk(1)
            words = self.word2vec.lang.index2word[topi.item()]
            result.append(words)
            # stopping criterion
            count += 1
            if count == limit or word == limit or count == 50 : stop = True
        print(' '.join(result + ['\033[0m']))
        return
    
    def generatePackedSentences(self, sentences, batch_size = 32) :
        sentences.sort(key = lambda s: len(s[0].split(' ')), reverse = True)
        packed_data = []
        for i in range(0, len(sentences), batch_size) :
            pack0 = [s[0].split(' ') for s in sentences[i:i + batch_size]]
            pack0 = [[self.word2vec.lang.getIndex(w) for w in words] for words in pack0]
            pack0 = [[w for w in words if w is not None] for words in pack0]
            pack0.sort(key = len, reverse = True)
            lengths = torch.tensor([len(p) for p in pack0])           # size = (batch_size) 
            pack0 = list(itertools.zip_longest(*pack0, fillvalue = self.word2vec.lang.getIndex('PADDING_WORD')))
            pack0 = Variable(torch.LongTensor(pack0).transpose(0, 1)) # size = (batch_size, max_length)
            pack1 = [[el[1]] for el in sentences[i:i + batch_size]]
            pack1 = Variable(torch.FloatTensor(pack1))                # size = (batch_size)       
            packed_data.append([[pack0, lengths], pack1])
        return packed_data
    
    def fit(self, batches, iters = None, epochs = None, lr = 0.025, random_state = 42,
              print_every = 10, compute_accuracy = True):
        """Performs training over a given dataset and along a specified amount of loops"""
        def asMinutes(s):
            m = math.floor(s / 60)
            s -= m * 60
            return '%dm %ds' % (m, s)

        def timeSince(since, percent):
            now = time.time()
            s = now - since
            rs = s/percent - s
            return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
        
        def computeLogProbs(batch) :
            embeddings = self.word2vec.embedding(batch[0].to(self.device))
            _, hidden  = self.context(embeddings, lengths = batch[1].to(self.device))
            log_probs  = F.log_softmax(self.out(hidden.view(batch[0].size(0), -1))) # dim = (batch_size, lang_size)
            return log_probs

        def computeAccuracy(log_probs, targets) :
            return sum(torch.abs(targets.view(-1) - self.act(log_probs)) < 0.5).item() * 100 / targets.size(0)

        def printScores(start, iter, iters, tot_loss, tot_loss_words, print_every, compute_accuracy) :
            avg_loss = tot_loss / print_every
            avg_loss_words = tot_loss_words / print_every
            if compute_accuracy : print(timeSince(start, iter / iters) + ' ({} {}%) loss : {:.3f}  accuracy : {:.1f} %'.format(iter, int(iter / iters * 100), avg_loss, avg_loss_words))
            else                : print(timeSince(start, iter / iters) + ' ({} {}%) loss : {:.3f}                     '.format(iter, int(iter / iters * 100), avg_loss))
            return 0, 0

        def trainLoop(batch, optimizer, compute_accuracy = True):
            """Performs a training loop, with forward pass, backward pass and weight update."""
            optimizer.zero_grad()
            self.zero_grad()
            log_probs = computeLogProbs(batch[0])
            targets   = batch[1].to(self.device).view(-1)
            loss      = self.criterion(log_probs, targets)
            loss.backward()
            optimizer.step() 
            accuracy = computeAccuracy(log_probs, targets) if compute_accuracy else 0
            return float(loss.data[0] / targets.size(0)), accuracy
        
        # --- main ---
        self.train()
        np.random.seed(random_state)
        start = time.time()
        optimizer = self.optimizer([param for param in self.parameters() if param.requires_grad == True], lr = lr)
        tot_loss = 0  
        tot_acc  = 0
        if epochs is None :
            for iter in range(1, iters + 1):
                batch = random.choice(batches)
                loss, acc = trainLoop(batch, optimizer, compute_accuracy)
                tot_loss += loss
                tot_acc += acc      
                if iter % print_every == 0 : 
                    tot_loss, tot_acc = printScores(start, iter, iters, tot_loss, tot_acc, print_every, compute_accuracy)
        else :
            iter = 0
            iters = len(batches) * epochs
            for epoch in range(1, epochs + 1):
                print('epoch ' + str(epoch))
                np.random.shuffle(batches)
                for batch in batches :
                    loss, acc = trainLoop(batch, optimizer, compute_accuracy)
                    tot_loss += loss
                    tot_acc += acc 
                    iter += 1
                    if iter % print_every == 0 : 
                        tot_loss, tot_acc = printScores(start, iter, iters, tot_loss, tot_acc, print_every, compute_accuracy)
        return

### Training

In [425]:
classifier = SentenceClassifier(twin_fastText_word2vec,
                                hidden_dim = 75, 
                                n_layers = 1, 
                                n_class = 2, 
                                dropout = 0.25,
                                criterion = nn.BCEWithLogitsLoss(size_average = False),
                                optimizer = optim.SGD,
                                device = device)

classifier.nbParametres()

91351

In [426]:
batches = classifier.generatePackedSentences(labelled_sentences, batch_size = 64)

#### Evaluation

#### Save

In [56]:
#torch.save(skipgram.state_dict(), path_to_NLP + '\\saves\\DDL_I1_skipgram.pth')